#### Init database

In [1]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode
from datetime import datedatetime, timedelta

import datetime

DB_NAME = 'yolov5'

TABLES = {}
# id  image_folder_path    image_name  detect_folder  detect_image  detect_detail predict_json  predict_date

TABLES['Image'] = (
    "CREATE TABLE `Image` ("
    "  `name`         varchar(255) NOT NULL,"
    "  `folder_path`  varchar(255) NOT NULL,"
    "   PRIMARY KEY(name, folder_path)"
    ") ENGINE=InnoDB"
)

TABLES['Predict_result'] = (
    "CREATE TABLE `Predict_result` ("
    "  `id` int NOT NULL AUTO_INCREMENT,"
    "  `image_name`   varchar(255) NOT NULL,"
    "  `image_folder` varchar(255) NOT NULL,"
    "  `predict_folder` varchar(255) NOT NULL,"
    "  `predict_image` varchar(255) NOT NULL,"
    "  `predict_json_file` varchar(255) NOT NULL,"
    "  `predict_json_str` JSON NOT NULL,"
    "  `time` DATETIME NOT NULL,"
    # "  `predict_date` date NOT NULL,"
    "  PRIMARY KEY (`id`),"
    # "   FOREIGN KEY (image_name)    REFERENCES Image(name),"
    "   FOREIGN KEY (image_name, image_folder)  REFERENCES Image(name,folder_path)"
    ") ENGINE=InnoDB")

cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="luan"
)

cursor = cnx.cursor()

In [2]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)
else:
    print('use successfull')

use successfull


In [3]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating table Image: OK
Creating table Predict_result: OK


In [4]:
cursor.close()

True

#### Detect

In [21]:
# from __future__ import print_function

# import mysql.connector
# from mysql.connector import errorcode
# from datetime import date, datetime, timedelta
# import datetime

# import torch
# import json

# cnx = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   password="luan",
#   database='yolov5'
# )

# cursor = cnx.cursor()

# add_output_th = ("INSERT INTO output_th "
#                "(folder_path, image_name, predict_json, predict_date) "
#                "VALUES (%(image_folder_path)s, %(image_name)s, %(predict_folder_path)s, %(predict_json)s, %(predict_date)s)")



In [1]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode
from datetime import date, datetime, timedelta
# import datetime

from pathlib import Path
import torch
import json

cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="luan",
  database='yolov5'
)

cursor = cnx.cursor()

# add_output_th = ("INSERT INTO output_th "
#                "(folder_path, image_name, predict_json, predict_date) "
#                "VALUES (%(image_folder_path)s, %(image_name)s, %(predict_folder_path)s, %(predict_json)s, %(predict_date)s)")

image_folder_path = 'datasets/data_th/images/val/'
image_name = 'THM_254.jpg'

predict_folder = 'predict/'
path_weights = 'yolov5/runs/train/exp9/weights/best.pt'


In [31]:



def detect_db(_img_name, _cursor:mysql.connector.cursor_cext.CMySQLCursor, _cnx: mysql.connector.connection_cext.CMySQLConnection, _folder = 'datasets/data_th/images/val/', _out_folder = 'predict/', _path_weights = 'yolov5/runs/train/exp9/weights/best.pt'):
    model = torch.hub.load('ultralytics/yolov5', 'custom', _path_weights)  # or yolov5m, yolov5l, yolov5x, etc.
    results = model(_folder + _img_name)
    # results.print()  # or .show(), .save(), .crop(), .pandas(), etc.
    now = datetime.now()
    formatted_date = now.strftime('%y%m%d_%Hh%Mm%Ss')
    formatted_date_db = now.strftime('%Y-%m-%d %H:%M:%S')
    predict_json_file = ''.join(map(str,image_name.split('.')[:-1])) + '-' + formatted_date + '.json'

    image_predict = results.pandas().xyxy[0]
    image_predict.to_json(_out_folder + predict_json_file, orient="records", indent=4)
    predict_json_str = image_predict.to_json(orient="records")
    
    predict_image = 'out_' + ''.join(map(str,image_name.split('.')[:-1])) + '-' + formatted_date + '.jpg'
    ################################################################################################
    pprint=False
    show=False
    save=True
    render=False
    labels=True
    # save_dir= Path('')
    save_dir= Path(_out_folder)
    # print('dir: ',save_dir)

    results.files = [predict_image]

    crops = []
    for i, (im, pred) in enumerate(zip(results.imgs, results.pred)):
        print(i)
        
        s = f'image {i + 1}/{len(results.pred)}: {im.shape[0]}x{im.shape[1]} '  # string
        if pred.shape[0]:
            for c in pred[:, -1].unique():
                n = (pred[:, -1] == c).sum()  # detections per class
                s += f"{n} {results.names[int(c)]}{'s' * (n > 1)}, "  # add to string
            if show or save or render:
                annotator = Annotator(im, example=str(results.names))
                for *box, conf, cls in reversed(pred):  # xyxy, confidence, class
                    label = f'{results.names[int(cls)]} {conf:.2f}'
                    annotator.box_label(box, label if labels else '', color=colors(cls))
                im = annotator.im
        else:
            s += '(no detections)'

        im = Image.fromarray(im.astype(np.uint8)) if isinstance(im, np.ndarray) else im  # from np
        if pprint:
            print(s.rstrip(', '))
        if show:
            im.show(results.files[i])  # show
        if save:
            f = results.files[i]
            im.save(save_dir / f)  # save
            if i == results.n - 1:
                print(f"Saved {results.n} image{'s' * (results.n > 1)} to {colorstr('bold', save_dir)}")
        if render:
            results.imgs[i] = np.asarray(im)
    ################################################################################################
    
    


    #   Insert to database
    #       Check is contain image
    query_name_img = "select * from Image where name = %s and folder_path = %s"
    _cursor.execute(query_name_img, (_img_name,_folder,))
    record = _cursor.fetchall()
    
    if record == []:
        cmd_insert_img = ("INSERT INTO yolov5.Image(name, folder_path) "
        # "VALUES({}, {})").format('12', '123123')
        "VALUES(\'{}\', \'{}\')").format(str(_img_name), str(_folder))
        
        # cmd_insert_img = "INSERT INTO yolov5.Image(name, folder_path) VALUES('15555555552', '132323')"
        # print('line 29: ', cmd_insert_img)
        
        _cursor.execute(cmd_insert_img)
        _cnx.commit()
    
    cmd_insert_predict = ("INSERT INTO Predict_result "
               "(image_name, image_folder, predict_folder, predict_image, predict_json_file, predict_json_str, time)"
               "VALUES (%s, %s, %s, %s, %s, %s, %s)")
    tmp_data = (_img_name, _folder, _out_folder, predict_image, predict_json_file, predict_json_str, formatted_date_db)
    _cursor.execute(cmd_insert_predict, tmp_data)
    _cnx.commit()
    
    #       Insert predict output to database
detect_db(image_name, cursor, cnx)

Using cache found in C:\Users\OS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-6-24 Python-3.9.0 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce MX130, 2048MiB)

YOLOv5  2022-6-24 Python-3.9.0 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce MX130, 2048MiB)

Fusing layers... 
Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients
Model summary: 213 layers, 7020913 parameters, 0 gradients
Adding AutoShape... 
Adding AutoShape... 


0
Saved 1 image to predict


In [ ]:
add_output_th = ("INSERT INTO output_th "
               "(image_folder_path, image_name, predict_folder_path, predict_json, predict_date)"
               "VALUES (%s, %s, %s, %s, %s)")


data_predict = (image_folder_path, image_name, predict_folder_path, tj, date_now,)

cursor.execute(add_output_th, data_predict)

cnx.commit()



In [12]:
from datetime import datetime
now = datetime.now()
formatted_date = now.strftime('%Y-%m-%d %H:%M:%S')

formatted_date

'2022-07-27 21:08:47'

#### Test

In [10]:
import mysql.connector

cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="luan",
  database='yolov5'
)

cursor = cnx.cursor()

sql_select_Query = "select * from output_th"

cursor.execute(sql_select_Query)
# get all records
records = cursor.fetchall()

print (records)

for rows in records:
    print(rows)

[(1, 'datasets/data_th/images/val/', 'THM_254.jpg', 'predict/', 'THM_25417h11.json', datetime.date(2022, 7, 26)), (2, 'datasets/data_th/images/val/', 'THM_254.jpg', 'predict/', 'THM_25417h22.json', datetime.date(2022, 7, 26))]
(1, 'datasets/data_th/images/val/', 'THM_254.jpg', 'predict/', 'THM_25417h11.json', datetime.date(2022, 7, 26))
(2, 'datasets/data_th/images/val/', 'THM_254.jpg', 'predict/', 'THM_25417h22.json', datetime.date(2022, 7, 26))


In [14]:
record2 = cursor.fetchall()

In [20]:
from datetime import datetime
now = datetime.now()
formatted_date = now.strftime('%Y-%m-%d %H:%M:%S')


formatted_date

'2022-07-28 01:16:31'

In [ ]:
("INSERT INTO Predict_result "
"(image_name, image_folder, predict_folder, predict_image, predict_json_file, predict_json_str, time)"
" VALUES {}-{}").format('rewr',12)

'INSERT INTO Predict_result (image_name, image_folder, predict_folder, predict_image, predict_json_file, predict_json_str, time) VALUES rewr-12'

In [29]:
from yolov5.utils.general import (LOGGER, check_requirements, check_suffix, check_version, colorstr, increment_path,
                           make_divisible, non_max_suppression, scale_coords, xywh2xyxy, xyxy2xywh)

from yolov5.utils.plots import Annotator, colors, save_one_box
from PIL import Image
import torch
import numpy as np
from pathlib import Path

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, etc.
# model = torch.hub.load('ultralytics/yolov5', 'custom', 'path/to/best.pt')  # custom trained model

print(type(model))

# Images
im = 'https://ultralytics.com/images/zidane.jpg'  # or file, Path, URL, PIL, OpenCV, numpy, list

# Inference
results = model(im)
####################################################################################################


####################################################################################################
# Results
# results.save('gg.jpg')  # or .show(), .save(), .crop(), .pandas(), etc.

results.xyxy[0]  # im predictions (tensor)
results.pandas().xyxy[0]  # im predictions (pandas)
#      xmin    ymin    xmax   ymax  confidence  class    name
# 0  749.50   43.50  1148.0  704.5    0.874023      0  person
# 2  114.75  195.75  1095.0  708.0    0.624512      0  person
# 3  986.00  304.00  1028.0  420.0    0.286865     27     tie

Using cache found in C:\Users\OS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-6-24 Python-3.9.0 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce MX130, 2048MiB)

YOLOv5  2022-6-24 Python-3.9.0 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce MX130, 2048MiB)

Fusing layers... 
Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Adding AutoShape... 


<class 'models.common.AutoShape'>
['phamthuha2.jpg']
0
Saved 1 image to predict
0 enddddd


,xmin,ymin,xmax,ymax,confidence,class,name
0,743.290527,48.343781,1141.756470,720.000000,0.879860,0,person
1,441.989594,437.336670,496.585114,710.036255,0.675119,27,tie
2,123.051056,193.238037,714.690674,719.771301,0.666694,0,person
3,978.989807,313.579468,1025.302856,415.526184,0.261517,27,tie


In [24]:
import pandas as pd
import numpy as np
import matplotlib as mpl

# df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
#                   index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
#                   columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))


df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns= pd.Index(['col1', 'col2', 'col3', 'col4', 'col5', 'col6'], name='col name'))



df.style


col name,col1,col2,col3,col4,col5,col6
Actual Label:,,,,,,
Tumour (Positive),38.000000,2.000000,18.000000,22.000000,21,nan
Non-Tumour (Negative),19.000000,439.000000,6.000000,452.000000,226,232.000000


In [25]:
a = df.to_json(orient="records", indent=4)

print(a)



a = df.to_json(orient="", indent=4)
print(a)

[
    {
        "col1":38.0,
        "col2":2.0,
        "col3":18.0,
        "col4":22.0,
        "col5":21,
        "col6":null
    },
    {
        "col1":19.0,
        "col2":439.0,
        "col3":6.0,
        "col4":452.0,
        "col5":226,
        "col6":232.0
    }
]


In [58]:
import builtins

import numpy as np

a = np.zeros((2,5,))

# a = a.tolist()
type(a)


a[0] = np.full((5, ), 8) 

for i, c in enumerate(a):
    print(i)
    print(c)
    
    
b = [1,4,5,6]
b = np

0
[8. 8. 8. 8. 8.]
1
[0. 0. 0. 0. 0.]
